#### Fold change of m6A ratio

In [7]:
#### pick out FUS-targets m6As and Non-targets m6As
os.chdir("/Charles/project/ASm6A/FUS_regulate/m6A_peak/")
os.system("mkdir -p targets")
os.system("mkdir -p non-targets")
##########
fus_1 = "/home/galaxy/data/RBP/POSTAR/POSTAR_bed/FUS_HEK293T_hg19.bed"
fus_2 = "/home/galaxy/data/RBP/POSTAR/POSTAR_bed/FUS_brain_hg19.bed"
fus_3 = "/home/galaxy/data/RBP/ENCODE/20201124_2121/merged_bed/hg19/rename/union_peaks/FUS.bed"
fus_4 = "/home/cjr/caoshuo/data/oRNAment/all_RBPs/hg19/FUS.bed"
fus_list = [fus_1, fus_2, fus_3, fus_4]

# def overlap_gene():
#     gene_withPromoter = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_GeneTSS2kb-Genend.bed"
#     fus_target_genes  = "fus_target_genes.bed"
#     # for fus in fus_list:
#     combined_fus = "combined_fus.bed"
#     # ! cat $fus_1 $fus_2 $fus_3 $fus_4 | awk '{OFS="\t"}{print $1,$2,$3}' - > combined_fus.bed
#     os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" % (gene_withPromoter,combined_fus,fus_target_genes))

# overlap_gene()
fus_target_genes  = "fus_target_genes.bed" # generated above
#########
m6a_dir = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/"
# ratio_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/peak_quan/spikeIn/foldChange/"
ratio_dir = "/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/peak_quan/totalReads/foldChange/"
ratio_list = glob.glob("%s/*.bed" % ratio_dir)

########## pick out m6A contained in wt samples.
def overlap_wtSamples(bed1, bed2):
    m6a_file = os.path.basename(bed1)
    os.system("bedtools intersect -a %s -b %s -wa > %s" % (bed1, bed2, m6a_file))
    return m6a_file

for ratio in ratio_list:
    wt_sample = os.path.basename(ratio).split("_")[1].split(".")[0]
    wt_m6a =  os.path.join(m6a_dir, "%s/ConsensusPeaks.bed" % wt_sample)
    ##
    m6a_bed = overlap_wtSamples(ratio, wt_m6a)
    res = "targets/%s" % m6a_bed
    os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" % (m6a_bed, fus_1, res))
    res = "non-targets/%s" % m6a_bed
    os.system("bedtools intersect -a %s -b %s -v -wa | sort -u > %s" % (m6a_bed, fus_target_genes, res))

In [8]:
#### pick out non-FUS targets (using raw m6A peaks)
result_dir = "/Charles/project/ASm6A/FUS_regulate/m6A_peak/non-target-rawm6A/"
m6a_dir = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/"
bed_list = glob.glob("%s/*/ConsensusPeaks.bed" % m6a_dir)

for bed in bed_list:
    prefix = bed.split("/")[-2]
    res = os.path.join(result_dir, "%s.bed" % prefix)
    os.system("bedtools intersect -a %s -b %s -v -wa | sort -u > %s" % (bed, fus_target_genes, res))

In [9]:
#### for boxplot (FUS target vs Non targets)
os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/peak_quan/spikeIn/")
matrix = "m6A_level.txt"
df = pd.read_table(matrix)

result_dir = "foldChange"
os.system("mkdir -p %s" % result_dir)

df = pd.read_table(in_file)
for col_x in ['wt1_ratio', 'wt2_ratio']:
# for col_x in ['pre-wt_m6ALevel','pre-wt2_m6ALevel','wt1_m6ALevel', 'wt2_m6ALevel']:
    for col_y in ['kd1-1_ratio', 'kd1-2_ratio', 'kd2-1_ratio', 'kd2-2_ratio']:
#     for col_y in ['pre-kd_m6ALevel', 'pre-kd2_m6ALevel','kd1-1_m6ALevel', 'kd1-2_m6ALevel', 'kd2-1_m6ALevel', 'kd2-2_m6ALevel']:
        col_list = ['#chro', 'start', 'end', col_x, col_y]
        df_sub = df[col_list].dropna()
        df_sub = df_sub[df_sub != np.inf] ####
        res = os.path.join(result_dir, "%s_%s.bed" % (col_x, col_y))
        df_sub.to_csv(res, sep="\t", index=False)

,#chro,start,end,wt1_ratio,wt2_ratio,kd1-1_ratio,kd1-2_ratio,kd2-1_ratio,kd2-2_ratio
0,chr1,14411,14829,11.449180,10.612748,9.053370,7.740901,8.978968,9.636243
1,chr1,788751,789941,30.889809,44.116044,22.335989,17.878844,34.456353,39.045098
2,chr1,791535,792683,19.558242,15.588879,21.728088,27.364667,12.496772,23.306706
3,chr1,847684,848533,33.386433,28.147125,15.988529,24.941173,23.451420,76.846168
4,chr1,854814,856396,14.325781,9.016192,13.538185,8.747803,10.941874,12.170612
